# Useful packages

In [1]:
import matplotlib.pyplot as plt

# Loading Steinmetz SPIKING data

In [2]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [3]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [4]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the fields below. For extra variables, check out the extra notebook and extra data files (lfp, waveforms and exact spike times, non-binned). 

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.    
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['ccf']`: Allen Institute brain atlas coordinates for each neuron. 
* `dat['ccf_axes']`: axes names for the Allen CCF. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_time']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: turning speed of the wheel that the mice uses to make a response, sampled at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position.
* `dat['face']`: average face motion energy from a video camera. 
* `dat['licks']`: lick detections, 0 or 1.   
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 
* `dat['prev_reward']`: time of the feedback (reward/white noise) on the previous trial in relation to the current stimulus time. 
* `dat['reaction_time']`: ntrials by 2. First column: reaction time computed from the wheel movement as the first sample above `5` ticks/10ms bin. Second column: direction of the wheel movement (`0` = no move detected).  




# Adding more trial identifiers into 'dat' (Tomáš)

The following four cells will derive **additional variables** about the stimuli and responses and will integrate them into the already existing `alldat` structure. See the text below the last cell for the variables' description.

In [5]:
#@title Wheel data extraction (made by Tomáš)
def extract_wheel_data(session, trial):
  """ Extracts the wheel speed and calculates the wheel position at each time bin. Requires the SPIKING dataset to be loaded.

  Args:
    session (int): For which session to extract the wheel data.
    trial (int): For which trial to extract the wheel data.
  
  Return:
    wheel_pos: A vector of positions sampled in 10 ms bins
    wheel_speed: A vector of speeds sampled in 10 ms bins
  """

  wheel_speed = alldat[session]['wheel'][0][trial]
  
  wheel_pos = np.zeros(len(wheel_speed))
  for t in range(1,len(wheel_speed)):
    wheel_pos[t] = wheel_pos[t-1] + wheel_speed[t]

  return wheel_pos, wheel_speed

In [6]:
#@title Stimulus data extraction (made by Tomáš)
def extract_stim_data(session):
  """ Extracts the stimulus information about each trial. Requires the SPIKING dataset to be loaded.

  Args:
    session (int): For which session to extract the stimulus data
    trial (int): For which trial to extract the stimulus data
  
  Return:
    stim_present(bool): A vector of booleans saying whether ANY stimulus was presented (True)
    stim_bilateral (bool): A vector of booleans saying whether BOTH stimuli were presented (True)
    stim_rightOnly (bool): A vector of booleans saying whether ONLY the RIGHT stimulus was presented (True)
    stim_leftOnly (bool): A vector of booleans saying whether ONLY the LEFT stimulus was presented (True)
    stim_diff (float): A vector of differences between the two stimuli contrasts; None means that only one stimulus was presented; positive values mean larger contrast in the right stimulus
  """

  right_contrast = alldat[session]['contrast_right']
  left_contrast = alldat[session]['contrast_left']

  stim_present = (right_contrast + left_contrast) > 0
  stim_bilateral = np.minimum(right_contrast, left_contrast) > 0
  
  stim_rightOnly = np.zeros_like(right_contrast)
  for i in range(len(right_contrast)):
    stim_rightOnly[i] = right_contrast[i] > 0 and left_contrast[i] == 0
  
  stim_leftOnly = np.zeros_like(left_contrast)
  for i in range(len(left_contrast)):
    stim_leftOnly[i] = left_contrast[i] > 0 and right_contrast[i] == 0
  
  stim_diff = right_contrast - left_contrast
  for i in range(len(stim_diff)):
     if stim_bilateral[i] == False:
       stim_diff[i] = None

  return stim_present, stim_bilateral, stim_rightOnly, stim_leftOnly, stim_diff

In [7]:
#@title Accuracy data extraction (made by Phil)
def compute_accuracy(data = alldat, session = list(np.arange(0, 39, 1))):
    """ 
    arguments:
    session: a list of session numbers to evaluate.  Default is all session in dat.
    
    output:
    TBD
    """
    accuracy_all = []
    accuracy_move_all = []
    accuracy_bool_all = []
    accuracy_bool_move_all = []
    indMove_all = []
    
    for i, run in enumerate(session):
        # get relevant data
        
        #set_trace()
        response = data[i]['response'] # right - nogo - left (-1, 0, 1)
        vis_right = data[i]['contrast_right'] # 0 - low - high
        vis_left = data[i]['contrast_left'] # 0 - low - high

        #initialize empty boolean accuracy object
        accuracy_bool = np.zeros(len(response))

        # the following are the correct responses:
        # if vis_left > vis_right : response >0
        # if vis_left < vis_right : response <0
        # if vis_left = vis_right : response =0
        # trials below red line are incorrect
        is_correct = np.sign(response)==np.sign(vis_left-vis_right)
        accuracy_bool[is_correct] = 1
        accuracy = np.sum(accuracy_bool)/len(accuracy_bool)

        # Compute accuracy for only movement trials
        indMove = np.logical_or(vis_right > vis_left, vis_left > vis_right)

        # create vectors only of movement trials
        response_move = response[indMove]
        vis_right_move = vis_right[indMove]
        vis_left_move = vis_left[indMove]

        # compute accuracy only for movement trials
        accuracy_bool_move = np.zeros(len(response_move))

        is_correct_move = np.sign(response_move)==np.sign(vis_left_move - vis_right_move)
        accuracy_bool_move[is_correct_move] = 1
        accuracy_move = np.sum(accuracy_bool_move)/len(accuracy_bool_move)

        #set_trace()
        accuracy_all.append(accuracy)
        accuracy_move_all.append(accuracy_move)
        accuracy_bool_all.append(accuracy_bool)
        accuracy_bool_move_all.append(accuracy_bool_move)
        indMove_all.append(indMove)
        
    return ([accuracy_all, accuracy_move_all, accuracy_bool_all, accuracy_bool_move_all, indMove])

In [8]:
#@title Integrating new variables into the alldat structure (made by Tomáš)
def make_trial_identifiers(data = alldat):
  """ 
  Updates the 'alldat' structure by adding new variables as keys into the
  dictionaries under each session. For dat=alldat[k], you will be able to
  access the added variables the same way as all other variables already present
  in the alldat structure.
  
  IMPORTANT: See the text under this cell for description of the new variables.
  """
  for session in range(len(data)):
    dat = data[session]
    n_trials = len(dat['response'])

    # Setting up wheel_pos and wheel_speed
    wheel_pos = np.empty((n_trials,250)) # hardcoded 250 as the universal number of timepoints in each trial
    wheel_speed = np.empty((n_trials,250)) # hardcoded 250 as the universal number of timepoints in each trial
    for trial in range(n_trials):
      this_wheel_pos, this_wheel_speed = extract_wheel_data(session, trial)
      wheel_pos[trial] = this_wheel_pos
      wheel_speed[trial] = this_wheel_speed
    dat['wheel_pos'] = wheel_pos
    dat['wheel_speed'] = wheel_speed # This is the same as dat['wheel']; included again with different name to avoid ambiguity
    
    # Setting up stim_present, stim_bilateral, stim_rightOnly, stim_leftOnly, and stim_diff
    stim_present, stim_bilateral, stim_rightOnly, stim_leftOnly, stim_diff = extract_stim_data(session)
    dat['stim_present'] = stim_present
    dat['stim_bilateral'] = stim_bilateral
    dat['stim_rightOnly'] = stim_rightOnly
    dat['stim_leftOnly'] = stim_leftOnly
    dat['stim_diff'] = stim_diff

    # Setting up response_side, response_correct
    dat['response_side'] = dat['response'] # This is the same as dat['response']; included again with different name to avoid ambiguity
    
    accuracy_all, accuracy_move_all, accuracy_bool_all, accuracy_bool_move_all, indMove = compute_accuracy()
    dat['accuracy_all'] = accuracy_all[session]
    dat['accuracy_moveOnly'] = accuracy_move_all[session]
    dat['response_correct_all'] = accuracy_bool_all[session]
    dat['response_correct_moveOnly'] = accuracy_bool_move_all[session]
    dat['movement_required'] = indMove[session]


make_trial_identifiers()

After running the `make_trial_identifiers()` function in the last cell, the `dat=alldat[k]` now contains these additional variables:

*   `dat['wheel_pos']`: trial-by-timepoint array of wheel positions
*   `dat['wheel_speed']`: trial-by-timepoint array of wheel speeds (similar to `dat['wheel']` but without the redundant dimension and ambiguous name)
*   `dat['stim_present']`: n_trials-long vector of booleans indicating whether **ANY** stimulus was presented in the given trial
*   `dat['stim_bilateral']`: n_trials-long vector of booleans indicating whether **BOTH** stimuli were presented in the given trial
*   `dat['stim_rightOnly']:` n_trials-long vector of booleans indicating whether **only the RIGHT** stimulus was presented
*   `dat['stim_leftOnly']`: n_trials-long vector of booleans indicating whether **only the LEFT** stimulus was presented
*   `dat['stim_diff']`: n_trials-long vector of differences between the two stimuli contrasts; *nan* means that only one stimulus was presented; positive values mean larger contrast in the right stimulus, negative values mean larger contrast in the left stimulus
*   `dat['response_side']`: n_trials-long vector of directions the mouse turned the wheel in (-1: counterclockwise, 1: clockwise, 0: no response); same as `dat['response']` but with unambiguous name
*   `dat['accuracy_all']`: percentage of correct responses of the mouse across all trials (one scalar per session)
*   `dat['accuracy_moveOnly']`: percentage of correct responses of the mouse for trials **requiring movement** only (one scalar per session)
*   `dat['response_correct_all']`: n_trials-long vector of correct responses (1) and incorrect responses (0) in all trials
*   `dat['response_correct_moveOnly']`: vector of variable length of correct responses (1) and incorrect responses (0) in trials **requiring movement** only
*   `dat['movement_required']`: n_trials-long vector of booleans indicating whether movement was required in the given trial